<a href="https://colab.research.google.com/github/nissimyo/DCGAN/blob/master/DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before using, you need to verify you have versions:
- TensorFlow: 2.3.0
- Keras: 2.4.3

In [ ]:
import tensorflow as tf
import keras as ke

print(tf.__version__)
print(ke.__version__)

### If needed

In [ ]:
pip install --upgrade tensorflow

In [ ]:
pip install --upgrade keras

In [18]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import scipy as sp
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
from scipy.signal import hilbert

import glob

from keras.models import Sequential
from keras.layers import Dense ,Conv1D, Flatten,Reshape,Conv1DTranspose
from keras.layers.advanced_activations import LeakyReLU


In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Reding file

In [21]:
dfs2_5 = pd.read_csv('/content/drive/My Drive/PHM DATA/XJTU/XJTU-SY_Bearing_Datasets/37.5Hz11kN/Bearing2_5/dfs2_5.csv')
dfs2_5 = dfs2_5.drop(dfs2_5.columns[0], axis=1)


## Discriminator

In [106]:
Signal_len = 4096

def create_conv_discriminator(Signal_len = 4096, optimizer = 'adam'):

  discriminator = Sequential()

  discriminator.add(Reshape((Signal_len, 1), input_shape=(Signal_len,)))

  discriminator.add(Conv1D(filters=32, kernel_size=25, activation='relu',input_shape=(Signal_len,1), strides=2))

  discriminator.add(Conv1D(filters=64, kernel_size=25, activation='relu',strides=2))

  discriminator.add(Conv1D(filters=128, kernel_size=25, activation='relu',strides=5))

  discriminator.add(Conv1D(filters=256, kernel_size=25, activation='relu',strides=5))

  discriminator.add(Flatten())

  discriminator.add(Dense(256, activation = 'relu'))
    
  discriminator.add(Dense(1, activation='sigmoid')) #Does not appear in the paper

  discriminator.compile(loss='binary_crossentropy', optimizer='adam')

  return discriminator


In [ ]:
dis = create_conv_discriminator()
dis.summary()

## Generator

In [108]:
import keras
from keras.models import Sequential
from keras.layers import Dense ,Conv2DTranspose , Reshape,MaxPooling2D,MaxPooling1D,Flatten,Conv1DTranspose

import tensorflow as tf

noise_dim = 100
def create_conv_generator(Signal_len = 4096, optimizer = 'adam'):

  generator = Sequential()      

  generator.add(Reshape((noise_dim, 1), input_shape=(noise_dim,)))

  generator.add(keras.layers.Conv1DTranspose(filters=128,kernel_size=25,strides=4,input_shape=(noise_dim,1),activation='relu'))

  generator.add(keras.layers.Conv1DTranspose(filters=64,kernel_size=25,strides=4,activation='relu'))

  generator.add(keras.layers.Conv1DTranspose(filters=32,kernel_size=25,strides=4,activation='relu'))

  generator.add(keras.layers.Conv1DTranspose(filters=1,kernel_size=25,strides=4,activation='tanh'))

  generator.add(Flatten())

  generator.add(Dense(4096)) #at the end?

  generator.compile(loss='binary_crossentropy', optimizer=optimizer)

  return generator

In [ ]:
gen = create_conv_generator()
gen.summary()

## Combine all

In [59]:
from keras.layers import Input
from keras.models import Model
noise_dim = 100
optimizer = 'adam'

dis.trainable = False

gan_input = Input(shape=(noise_dim,1,))
fake_signal = gen(gan_input)

gan_output = dis(fake_signal)

gan = Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer=optimizer)

## Training

In [74]:
## change to seperated signals
x_train = np.array(dfs2_5.iloc[0:180])
x_train = x_train.reshape(180*8,np.int(32768/8)) # Splitting the signals to 4096

In [ ]:
epochs = 100
batch_size = 10
steps_per_epoch = 10

##GAN - training with for loop
for epoch in range(epochs):
    for batch in range(steps_per_epoch):
        noise = np.random.normal(0, 1, size=(batch_size, noise_dim))
        fake_x = gen.predict(noise)

        real_x = x_train[np.random.randint(0, x_train.shape[0], size=batch_size)]
        
        x = np.concatenate((real_x, fake_x))

        disc_y = np.zeros(2*batch_size)
        disc_y[:batch_size] = 0.9

        #dis.trainable=True
        d_loss = dis.train_on_batch(x, disc_y)

        y_gen = np.ones(batch_size)

        #dis.trainable=False

        g_loss = gan.train_on_batch(noise, y_gen)

    print(f'Epoch: {epoch} \t Discriminator Loss: {d_loss} \t\t Generator Loss: {g_loss}')

In [ ]:
dis.predict(x_train[180*8].reshape(1,4096))[0]

In [ ]:
plt.figure(figsize=(20,13))
pred = gen.predict(np.random.normal(0,1,100).reshape(-1,100))[0]
plt.plot(pred)

In [ ]:
N = np.int(4096) # Number of sample points
T = 1.28/8
dT = T/N # sample spacing
#x = np.linspace(0.0, T-dT, N)

yf = sp.fft.fft(np.array(gen.predict(np.random.normal(0,1,100).reshape(-1,100))[0]))
xf = np.linspace(0.0, 1.0/(2.0*dT), N//2)
f = px.line(x=xf, y=np.log(2.0/N * np.abs(yf[0:N//2])))
f.show()

### Testing

In [ ]:
## change to seperated signals
x_test = np.array(dfs2_5.iloc[0:330])
x_test = x_test.reshape(330*8,np.int(32768/8)) # Splitting the signals to 4096
dis.predict(x_test)

In [ ]:
dis.predict(x_test[300*8].reshape(1,4096))